In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import keras.models
# from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot

# Carga un solo archivo como una matriz numpy
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

# carga una lista de archivos y regresa como una matriz numérica 3d
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # apila un grupo para que las características sean de 3 dimensiones
    loaded = dstack(loaded)
    return loaded

# carga el dataset de un grupo, entrenamiento o prueba
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # carga todos los archivos como un arreglo
    filenames = list()
    # aceleración sensor 1
    filenames += ['acc_x_1_'+group+'.txt', 'acc_y_1_'+group+'.txt', 'acc_z_1_'+group+'.txt']
    # giroscopio sensor 1
    filenames += ['gyro_x_1_'+group+'.txt', 'gyro_y_1_'+group+'.txt', 'gyro_z_1_'+group+'.txt']
    # aceleración sensor 2
    filenames += ['acc_x_2_'+group+'.txt', 'acc_y_2_'+group+'.txt', 'acc_z_2_'+group+'.txt']
    # giroscopio sensor 2
    filenames += ['gyro_x_2_'+group+'.txt', 'gyro_y_2_'+group+'.txt', 'gyro_z_2_'+group+'.txt']
    # carga los datos de entrada
    X = load_group(filenames, filepath)
    # carga las clases de salida
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

# carga los datasets, devuelve datos y etiquetas de entrenamiento y prueba
def load_dataset(prefix=''):
    # carga todo el entrenamiento
    trainX, trainy = load_dataset_group('train', prefix + 'SensorDataset/')
    print(trainX.shape, trainy.shape)
    # carga todo el test
    testX, testy = load_dataset_group('test', prefix + 'SensorDataset/')
    print(testX.shape, testy.shape)
    # valores de clase de compensación cero
    trainy = trainy - 1
    testy = testy - 1
    # codificación one hot en y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

def test(trainX, trainy, testX, testy,path='convlstm_sensors'):
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # reshape en subsecuencias (muestras, pasos de tiempo, filas, columnas, canales)
    n_steps, n_length = 10, 15
    trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
    model = keras.models.load_model(path+'.h5')
    # evaluar modelo
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    test_preds = np.argmax(model.predict(testX), axis=-1) # 0 significa aceptado, 1 rechazado
    return accuracy, test_preds

In [ ]:
trainX, trainy, testX, testy = load_dataset()
score, pred = test(trainX, trainy, testX, testy)
score * 100.0 , pred